In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import tensorflow as tf
import numpy as np

import time

import random
tf.random.set_seed(3407)
np.random.seed(3407)
random.seed(3407)

from sklearn.metrics import f1_score

from create_spectrogram import (
    create_spectrograms_from_audio_dataset, 
)
from helper_functions import (
    evaluate_prediction,
    get_file_size, 
    convert_bytes, 
    convert_prefetchdataset_to_numpy_arrays,
    predict_and_print_full_results,
    evaluate_time_of_prediction,
    full_int_model_predict,
    get_f1_scores_of_non_overlapping_partitions_full_int_q,
    get_f1_scores_of_bootstraping_partitions_full_int_q,
    )

In [15]:
train_dataset = tf.keras.utils.audio_dataset_from_directory("../dataset/training", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
test_dataset = tf.keras.utils.audio_dataset_from_directory("../dataset/testing", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
val_dataset = tf.keras.utils.audio_dataset_from_directory("../dataset/validation", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)

label_names = np.array(test_dataset.class_names)
print("Classes: ", label_names)

Found 11292 files belonging to 2 classes.


Found 1393 files belonging to 2 classes.


Found 1380 files belonging to 2 classes.


Classes:  ['non_target' 'target']


In [16]:
train_spectrogram_ds = create_spectrograms_from_audio_dataset(train_dataset, sample_rate = 16000).cache().prefetch(tf.data.AUTOTUNE)
test_spectrogram_ds = create_spectrograms_from_audio_dataset(test_dataset, sample_rate = 16000).cache().prefetch(tf.data.AUTOTUNE)
val_spectrogram_ds = create_spectrograms_from_audio_dataset(val_dataset, sample_rate = 16000).cache().prefetch(tf.data.AUTOTUNE)

x_train_np, y_train_np = convert_prefetchdataset_to_numpy_arrays(train_spectrogram_ds)
x_val_np, y_val_np = convert_prefetchdataset_to_numpy_arrays(val_spectrogram_ds)
x_test_np, y_test_np = convert_prefetchdataset_to_numpy_arrays(test_spectrogram_ds)

In [17]:
BASE_MODEL_FILE_NAME = "../spectrogram_models_from_notebooks/cnn/hpo/cnn_mel_spec_baseline.keras"
baseline_model = tf.keras.models.load_model(BASE_MODEL_FILE_NAME)
baseline_model.summary()
convert_bytes(get_file_size(BASE_MODEL_FILE_NAME), "MB")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 182, 78, 4)        40        
                                                                 
 max_pooling2d (MaxPooling2  (None, 91, 39, 4)         0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 89, 37, 4)         148       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 44, 18, 4)         0         
 g2D)                                                            
                                                                 
 reshape (Reshape)           (None, 3168)              0         
                                                                 
 dense (Dense)               (None, 8)                 2

In [18]:
print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
) = predict_and_print_full_results(baseline_model, x_test_np, y_test_np, model_format="keras")


Test dataset:
44/44 [==============================] - 1s 12ms/step
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.99%
Recall: 99.13%
Precision: 97.84%
F1-score: 98.48%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9844262846996378

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9817187973649888


### Convert to TF Lite format

In [7]:
converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
tflite_model = converter.convert()


INFO:tensorflow:Assets written to: /tmp/tmpw0zi4ybp/assets


INFO:tensorflow:Assets written to: /tmp/tmpw0zi4ybp/assets
2024-07-28 21:16:12.011245: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-07-28 21:16:12.011297: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-07-28 21:16:12.011620: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpw0zi4ybp
2024-07-28 21:16:12.013099: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-28 21:16:12.013115: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpw0zi4ybp
2024-07-28 21:16:12.016469: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-07-28 21:16:12.017488: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-07-28 21:16:12.055495: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

In [8]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(tflite_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(tflite_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(tflite_model, x_test_np, y_test_np, model_format="tf_lite")

TF_LITE_MODEL_FILE_NAME = "../spectrogram_models_from_notebooks/cnn/quantized/cnn_mel_spec.tflite"
open(TF_LITE_MODEL_FILE_NAME, "wb").write(tflite_model)
print("\n")
print("Model file name: ", TF_LITE_MODEL_FILE_NAME)
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "KB")

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 99.06%
Recall: 99.36%
Precision: 97.91%
F1-score: 98.63%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.986689172446992

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9861205880566085

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.99%
Recall: 99.13%
Precision: 97.84%
F1-score: 98.48%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9844262846996378

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9836996874048178

Time for Test dataset:
Accuracy: 98.99%
Recall: 99.13%
Precision: 97.84%
F1-score: 98.48%

Time to make a prediction for a single data point
Mean: 0.002 seconds
Max: 0.003 seconds
M

### Dynamic range quantization

In [9]:
dynamic_range_quant_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
dynamic_range_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
dynamic_range_quant_model = dynamic_range_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpknx1ul0u/assets


INFO:tensorflow:Assets written to: /tmp/tmpknx1ul0u/assets
2024-07-28 21:16:44.099368: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-07-28 21:16:44.099433: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-07-28 21:16:44.099639: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpknx1ul0u
2024-07-28 21:16:44.100525: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-28 21:16:44.100538: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpknx1ul0u
2024-07-28 21:16:44.103224: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-07-28 21:16:44.150355: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmpknx1ul0u
2024-07-28 21:16:44.163681: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [10]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(dynamic_range_quant_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(dynamic_range_quant_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(dynamic_range_quant_model, x_test_np, y_test_np, model_format="tf_lite")

DRQ_MODEL_FILE_NAME = "../spectrogram_models_from_notebooks/cnn/quantized/cnn_mel_spec_drq.tflite"
open(DRQ_MODEL_FILE_NAME, "wb").write(dynamic_range_quant_model)
print("\n")
print("Model file name: ", DRQ_MODEL_FILE_NAME)
convert_bytes(get_file_size(DRQ_MODEL_FILE_NAME), "KB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 99.06%
Recall: 99.36%
Precision: 97.91%
F1-score: 98.63%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.986689172446992

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.986917253178175

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.99%
Recall: 99.13%
Precision: 97.84%
F1-score: 98.48%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9844262846996378

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9842259233992772

Time for Test dataset:
Accuracy: 98.99%
Recall: 99.13%
Precision: 97.84%
F1-score: 98.48%

Time to make a prediction for a single data point
Mean: 0.002 seconds
Max: 0.003 seconds
Mi

### Float16 quantization

In [11]:
float16_quant_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
float16_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
float16_quant_converter.target_spec.supported_types = [tf.float16]
float16_quant_model = float16_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpzomhttbn/assets


INFO:tensorflow:Assets written to: /tmp/tmpzomhttbn/assets
2024-07-28 21:17:16.212965: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-07-28 21:17:16.213018: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-07-28 21:17:16.213193: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpzomhttbn
2024-07-28 21:17:16.214100: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-28 21:17:16.214113: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpzomhttbn
2024-07-28 21:17:16.216557: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-07-28 21:17:16.253767: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmpzomhttbn
2024-07-28 21:17:16.266078: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [12]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(float16_quant_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(float16_quant_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(float16_quant_model, x_test_np, y_test_np, model_format="tf_lite")

FLOAT16_QUANT_MODEL_FILE_NAME = "../spectrogram_models_from_notebooks/cnn/quantized/cnn_mel_spec_float16q.tflite"
open(FLOAT16_QUANT_MODEL_FILE_NAME, "wb").write(float16_quant_model)
print("\n")
print("Model file name: ", FLOAT16_QUANT_MODEL_FILE_NAME)
convert_bytes(get_file_size(FLOAT16_QUANT_MODEL_FILE_NAME), "KB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 99.06%
Recall: 99.36%
Precision: 97.91%
F1-score: 98.63%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.986689172446992

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9817247033115624

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.99%
Recall: 99.13%
Precision: 97.84%
F1-score: 98.48%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9844262846996378

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9835029887214596

Time for Test dataset:
Accuracy: 98.99%
Recall: 99.13%
Precision: 97.84%
F1-score: 98.48%

Time to make a prediction for a single data point
Mean: 0.002 seconds
Max: 0.004 seconds
M

### Float Fallback quantization

In [13]:
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(x_val_np).batch(1).take(100):
    yield [input_value]

float_fallback_quant_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
float_fallback_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
float_fallback_quant_converter.representative_dataset = representative_data_gen
float_fallback_quant_model = float_fallback_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpg5ecs3c3/assets


INFO:tensorflow:Assets written to: /tmp/tmpg5ecs3c3/assets
/home/polina/newname/.venv/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-07-28 21:17:48.142357: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-07-28 21:17:48.142413: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-07-28 21:17:48.142622: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpg5ecs3c3
2024-07-28 21:17:48.143795: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-28 21:17:48.143814: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpg5ecs3c3
2024-07-28 21:17:48.146665: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-07-28 21:17

In [14]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(float_fallback_quant_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(float_fallback_quant_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(float_fallback_quant_model, x_test_np, y_test_np, model_format="tf_lite")

FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME = "../spectrogram_models_from_notebooks/cnn/quantized/cnn_mel_spec_fallback_q.tflite"
open(FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME, "wb").write(float_fallback_quant_model)
print("\n")
print("Model file name: ", FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME)
convert_bytes(get_file_size(FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME), "MB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 99.20%
Recall: 98.94%
Precision: 98.73%
F1-score: 98.84%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9882782955717933

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9888495157514812

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.85%
Recall: 98.47%
Precision: 98.04%
F1-score: 98.26%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9823828598412743

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9796946307233569

Time for Test dataset:
Accuracy: 98.85%
Recall: 98.47%
Precision: 98.04%
F1-score: 98.26%

Time to make a prediction for a single data point
Mean: 0.001 seconds
Max: 0.003 seconds


### Full integer quantization

In [19]:
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(x_val_np).batch(1).take(5000):
    yield [input_value]

full_int_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
full_int_converter.optimizations = [tf.lite.Optimize.DEFAULT]
full_int_converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
full_int_converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
full_int_converter.inference_input_type = tf.uint8
full_int_converter.inference_output_type = tf.uint8

tflite_model_full_int = full_int_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpt84fdkb3/assets


INFO:tensorflow:Assets written to: /tmp/tmpt84fdkb3/assets
/home/polina/newname/.venv/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-08-28 18:39:38.436815: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-08-28 18:39:38.437554: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-08-28 18:39:38.440603: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpt84fdkb3
2024-08-28 18:39:38.442544: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-08-28 18:39:38.442566: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpt84fdkb3
2024-08-28 18:39:38.447904: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-08-28 18:39

In [20]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_full_int)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.uint8'>
output:  <class 'numpy.uint8'>


In [21]:
import pathlib

tflite_models_dir = pathlib.Path("../spectrogram_models_from_notebooks/cnn/quantized")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the quantized model:
tflite_model_full_int_file = tflite_models_dir/"cnn_mel_spec_full_int_q.tflite"
tflite_model_full_int_file.write_bytes(tflite_model_full_int)

30848

In [22]:
print("Validation dataset:")
predictions = full_int_model_predict(tflite_model_full_int_file, x_val_np)
evaluate_prediction(y_val_np, predictions)

print("\nDevide dataset into 10 non-overlapping patritions and get their mean F1-score")
non_overlap_patritions_f1_scores = get_f1_scores_of_non_overlapping_partitions_full_int_q(tflite_model_full_int_file, x_val_np, y_val_np)
print("Non-overlap mean F1-score: ", np.mean(non_overlap_patritions_f1_scores))

print("\nGet 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score")
bootstrap_patritions_f1_scores = get_f1_scores_of_bootstraping_partitions_full_int_q(tflite_model_full_int_file, x_val_np, y_val_np)
print("Bootstrap mean F1-score: ", np.mean(bootstrap_patritions_f1_scores))



print("\nTest dataset:")
predictions = full_int_model_predict(tflite_model_full_int_file, x_test_np)
evaluate_prediction(y_test_np, predictions)

print("\nDevide dataset into 10 non-overlapping patritions and get their mean F1-score")
non_overlap_patritions_f1_scores = get_f1_scores_of_non_overlapping_partitions_full_int_q(tflite_model_full_int_file, x_test_np, y_test_np)
print("Non-overlap mean F1-score: ", np.mean(non_overlap_patritions_f1_scores))

print("\nGet 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score")
bootstrap_patritions_f1_scores = get_f1_scores_of_bootstraping_partitions_full_int_q(tflite_model_full_int_file, x_test_np, y_test_np)
print("Bootstrap mean F1-score: ", np.mean(bootstrap_patritions_f1_scores))

print("\nTime for Test dataset:")
time_data = []
for data_point in x_test_np:
    start_time = time.time()
    predictions = full_int_model_predict(tflite_model_full_int_file, [data_point])
    elapsed_time = time.time() - start_time
    time_data.append(elapsed_time)
print("\nTime to make a prediction for a single data point")
print(f"Mean: {round(np.mean(time_data), 3)} seconds")
print(f"Max: {round(np.max(time_data), 3)} seconds")
print(f"Min: {round(np.min(time_data), 3)} seconds")

convert_bytes(get_file_size(tflite_model_full_int_file), "KB")

Validation dataset:
Accuracy: 98.99%
Recall: 98.52%
Precision: 98.52%
F1-score: 98.52%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9856393049618838

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9849218134985217

Test dataset:
Accuracy: 98.71%
Recall: 98.25%
Precision: 97.83%
F1-score: 98.04%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9799956481565243

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.978961040800781

Time for Test dataset:

Time to make a prediction for a single data point
Mean: 0.001 seconds
Max: 0.003 seconds
Min: 0.001 seconds
File size: 30.125 Kilobytes


'30.125 Kilobytes'

In [19]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_full_int)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.uint8'>
output:  <class 'numpy.uint8'>
